# Part 4: Logistic

In this part, you will complete a Do-It-Yourself (DIY) implementation of binary logistic regression in an object-oriented pattern that corresponds with the Scikit-Learn API.

**Learning objectives.** You will:
1. Write object-oriented code for a Python class, matching standard API patterns.
2. Apply numerical Python (NumPy) to efficiently implement binary logistic regression, including code to fit the model to data using the gradient descent algorithm. 
3. Evaluate your implementation compared to the Scikit-Learn standard on synthetic data. 
4. Perform an ablation study on the impact of the learning rate hyperparameter for fitting a logistic regression model.

## Task 1

First, we will use Scikit-Learn to develop a baseline logistic regression model to which we can compare our DIY implementation. Run the following code to generate synthetic data for use in this part of the assignment. Observe that the predictive target is coded as 0 or 1, that the `sigmoid` function is defined for you, and that the code also splits the synthetic data into train and test sets for you.

Use Scikit-Learn to fit a [logistic regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#logisticregression) on the train set with the parameter setting `penalty = 'None'` (this will train a basic model without applying any regularization). Evaluate and report the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of the model on both the train set and the test set.

In [1]:
# Run but do not modify this code

import numpy as np
from sklearn.model_selection import train_test_split

def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

np.random.seed(2024)
n = 1000
features = 20

X = np.random.normal(size = (n, features))
weights = np.random.normal(size = features)
probs = sigmoid(X @ weights + np.random.normal(scale=0.01)) 
y = np.random.binomial(n=1, p=probs)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(penalty=None, random_state=2024)
model.fit(X_train, y_train)

# Predict on both the train and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate the accuracy on both sets
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

Training accuracy: 0.8714285714285714
Test accuracy: 0.88


## Task 2

Complete the following class to implement binary logistic regression. Some important notes about the implementation:

1. Remember that the Scikit-Learn API treats an input `X` array, whether to `fit` or `predict`, as a matrix with a row for every data point and a column for every feature. 
2. For `fit`, every row in `X` corresponds to a given output in `y`, and  you don't need to return anything, just optimize the internal model weights (which should be stored as instance variables). For `predict_proba` and `predict`, you should return a NumPy array with one element (corresponding to a probability or a 0/1 value) for every row in the input `X`.
3. Remember that logistic regression models the probability of outputting `1` as a sigmoid of a linear function of features. This has several implications. One is that the number of weights in your model should equal the number of features equal to the number of columns of the `X` matrix passed to the `fit` method. We recommend that you initialize these weights as random normally distributed values, for example by using NumPy's [random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html). Another implication is that `predict_proba` should return the sigmoid activation of the [dot product](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) (multiply element-wise then add together) of your model's weights and the given input. You do **not** need to include a bias term for this implementation.
4. For the `predict` function, you should use a simple thresholding of 0.5. That is, you should calculate the probabilities using the `predict_proba` method and return `1` if the probability is greater than 0.5 and `0` otherwise. You can assume that `y` will consist exclusively of `0`s or `1`s for the purpose of this implementation.
5. The `fit` method should implement gradient descent on the log-likelihood. For a given feature/weight dimension $j$ and a particular data point $x$ with label $y$, the partial derivative with respect to weight $w_j$ is $(a - y)x_j$ where $a$ is the activation (the predicted probability) assosciated with example $x$. For each feature, this quantity should be averaged over all training data. The vector of all such values forms the gradient $\vec{\nabla}$. The gradient descent learning update should then be $\vec{w'} = \vec{w} - \eta \vec{\nabla}$ where $\eta$ is the learning rate `lr` passed to the constructor, $\vec{w}$ are the previous weights and $\vec{w'}$ are the weights for the next iteration. The algorithm should proceed for `iters` iterations.
6. You will note the `fit` method takes an optional `verbose` parameter. While it is not required, we highly recommend that you include code in the `fit` method that, when `verbose` is `True`, provides additional logging or printing of information about the training process to help debug. 
7. The `pass` statements are syntactic placeholders that should be removed when you implement a method.
8. Finally, note that your implementation will be much more efficient if you use vectorized NumPy operations and avoid for loops or nested for loops over large amounts of data.

In [4]:
import numpy as np

class BinaryLogisticRegression:
    def __init__(self, lr=0.1, iters=100, random_state=2024):
        self.lr = lr
        self.iters = iters
        self.random_state = random_state 

    def sigmoid(self, z):
        return 1.0/(1.0 + np.exp(-z))

    def predict_proba(self, X):
        return self.sigmoid(X @ self.weights)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5).astype(int)

    def fit(self, X, y, verbose=False):
        np.random.seed(self.random_state)
        self.weights = np.random.normal(size=X.shape[1])

        for i in range(self.iters):
            proba = self.predict_proba(X)
            gradient = np.dot(X.T, proba - y) / len(y)
            self.weights -= self.lr * gradient
            
            if verbose and i % 100 == 0:
                loss = -np.mean(y * np.log(proba) + (1 - y) * np.log(1 - proba))
                print(f"Iteration {i}: Loss = {loss}")

## Task 3

Use your DIY `BinaryLogisticRegression` class from task 2 to fit a logistic regression model on the train set as you did for the Scikit-Learn implementation in task 1. Use the default parameters (`lr=0.1, iters=1000, random_state=2024`). Evaluate and report the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of your DIY model on both the train set and the test set. You should achieve similar performance compared to the Scikit-Learn implementation.

In [6]:
from sklearn.metrics import accuracy_score

my_model = BinaryLogisticRegression(lr=0.1, iters=1000, random_state=2024)
my_model.fit(X_train, y_train)

y_train_pred = my_model.predict(X_train)
y_test_pred = my_model.predict(X_test)

training_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training accuracy (DIY logistic regression): {train_accuracy}")
print(f"Test accuracy (DIY logistic regression): {test_accuracy}")

Training accuracy (DIY logistic regression): 0.8671428571428571
Test accuracy (DIY logistic regression): 0.8766666666666667


## Task 4

Perform an *ablation* study on the learning rate hyperparameter `lr`. Specifically, try fitting 20 different models using your DIY `BinaryLogisticRegression` implementation, trying every combination of parameter settings `[10, 1, 0.1, 0.01, 0.001]` for the learning rate `lr` and `[1, 5, 20, 100]` for the number of gradient descent iterations `iters`. For each combination, evaluate the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of the model on the train set only (note that hyperparameters should never be selected using the test data).

Report all of your accuracies in a clearly labeled [Markdown table](https://www.markdownguide.org/extended-syntax/#tables). For readability, only report three digits per entry; for example, if the accuracy of a run is `0.8747340`, write `0.874` or `87.4` in the table. Explain which setting of `lr` you would use in this case and why.

In [10]:
import pandas as pd

lrs = [10, 1, 0.1, 0.01, 0.001] 
iters_list = [1, 5, 20, 100]
results = []

for lr in lrs:
    for iters in iters_list:
        my_model = BinaryLogisticRegression(lr=lr, iters=iters, random_state=2024)
        my_model.fit(X_train, y_train)
        y_train_pred = my_model.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        results.append([lr, iters, round(train_accuracy, 3)])

results_df = pd.DataFrame(results, columns=["Learning Rate (lr)", "Iterations (iters)", "Train Accuracy"])
print(results_df)

    Learning Rate (lr)  Iterations (iters)  Train Accuracy
0               10.000                   1           0.841
1               10.000                   5           0.870
2               10.000                  20           0.869
3               10.000                 100           0.867
4                1.000                   1           0.629
5                1.000                   5           0.716
6                1.000                  20           0.850
7                1.000                 100           0.866
8                0.100                   1           0.617
9                0.100                   5           0.620
10               0.100                  20           0.654
11               0.100                 100           0.786
12               0.010                   1           0.614
13               0.010                   5           0.616
14               0.010                  20           0.617
15               0.010                 100           0.6

The table suggests that a learning rate of 10 maximizes training accuracy, regardless of the number of iterations. The training accuracy of every other value for lr pales in comparison to that of lr=10, so that is the value I'll select here. 